In [1]:
import yaml
with open('Config.yml', 'r', encoding='utf-8') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
config

{'ModelName': 'qwen2.5:3b',
 'SystemPath': {'Programs': {'Paths': ['C:\\Users\\Administrator\\AppData\\Roaming\\Microsoft\\Windows\\Start Menu',
    'C:\\ProgramData\\Microsoft\\Windows\\Start Menu\\Programs'],
   'Exclude': {'Name': '卸载,Installer, Help,配置工具,Uninstall,Release Notes,FAQ,ReadMe,User’s Guide',
    'Path': 'Windows Kits,Git,MySQL Server 8.0,Stardock,Tesseract-OCR,Visual Studio 2022'}}}}

In [2]:
import os
def get_files(folder_paths):
    pgo = {}
    for folder_path in folder_paths:
        for filepath,dirnames,filenames in os.walk(folder_path):
            for filename in filenames:
                if filename.endswith('.lnk') or filename.endswith('.exe'):
                    name = filename.split('.')[0]
                    flg = True
                    for key in config['SystemPath']['Programs']['Exclude']['Name'].split(','):
                        if key in name or key == name:
                            flg = False
                            break
                    if flg:
                        for key in config['SystemPath']['Programs']['Exclude']['Path'].split(','):
                            if key in name or key == name:
                                flg = False
                                break
                    if flg:
                        pgo[name] = os.path.join(filepath, filename)
    return pgo

In [ ]:
folder_path = 'path/to/your/folder'
get_files(config['SystemPath']['Programs']['Paths'])

In [8]:
pgo

{'Administrative Tools': 'C:\\Users\\Administrator\\AppData\\Roaming\\Microsoft\\Windows\\Start Menu\\Programs\\Administrative Tools.lnk',
 'File Explorer': 'C:\\Users\\Administrator\\AppData\\Roaming\\Microsoft\\Windows\\Start Menu\\Programs\\File Explorer.lnk',
 'IDEA': 'C:\\Users\\Administrator\\AppData\\Roaming\\Microsoft\\Windows\\Start Menu\\Programs\\IDEA.lnk',
 'malf': 'C:\\Users\\Administrator\\AppData\\Roaming\\Microsoft\\Windows\\Start Menu\\Programs\\malf.lnk',
 'Ollama': 'C:\\Users\\Administrator\\AppData\\Roaming\\Microsoft\\Windows\\Start Menu\\Programs\\Startup\\Ollama.lnk',
 'PC Health Check': 'C:\\Users\\Administrator\\AppData\\Roaming\\Microsoft\\Windows\\Start Menu\\Programs\\PC Health Check.lnk',
 'WPS Office': 'C:\\Users\\Administrator\\AppData\\Roaming\\Microsoft\\Windows\\Start Menu\\Programs\\WPS Office.lnk',
 'LiveCaptions': 'C:\\Users\\Administrator\\AppData\\Roaming\\Microsoft\\Windows\\Start Menu\\Programs\\Accessibility\\LiveCaptions.lnk',
 'Magnify': 'C:\

In [6]:
import tools.tool_config as tool_config


tools = tool_config.tools
tools[2]['function']['parameters']['properties']['software_name']['enum'] = list(pgo.keys())


In [7]:
from ollama_qw import OllamaQW
import ollama
def main():
    qw = OllamaQW(model_name='qwen2.5:3b',tools=tools)
    while True:
        msg = input(">: ")
        if msg.lower() == 'exit':
            break
        res = qw.chat(msg)
        print(f"Alice: {res['content']}")
main()

2025-01-13 10:36:20.031 | INFO     | ollama_qw:__init__:21 - [{'type': 'function', 'function': {'name': 'close_light', 'description': '关掉机箱的灯。', 'parameters': {'type': 'object', 'properties': {}, 'required': []}}}, {'type': 'function', 'function': {'name': 'open_light', 'description': '打开机箱的灯。', 'parameters': {'type': 'object', 'properties': {}, 'required': []}}}, {'type': 'function', 'function': {'name': 'run_software', 'description': '根据给出的名称去运行指定的程序软件或者脚本等可执行的。', 'parameters': {'type': 'object', 'properties': {'software_name': {'type': 'string', 'enum': ['Administrative Tools', 'File Explorer', 'IDEA', 'malf', 'Ollama', 'PC Health Check', 'WPS Office', 'LiveCaptions', 'Magnify', 'Narrator', 'On-Screen Keyboard', 'VoiceAccess', 'LetsVPN', 'MongoDBCompass', 'Pandoc User’s Guide', 'Postman', 'Raspberry Pi Imager', 'Steam', 'Command Prompt', 'Control Panel', 'Run', 'Ubisoft Connect', 'Uplay', 'Windows PowerShell (x86)', 'Windows PowerShell', '百度网盘', 'Microsoft Edge', 'Motrix', 'Tabby Te

Running software QQ


2025-01-13 10:36:40.990 | INFO     | ollama_qw:chat:55 - model response is: model='qwen2.5:3b' created_at='2025-01-13T02:36:40.9888912Z' done=True done_reason='stop' total_duration=1050343900 load_duration=29899900 prompt_eval_count=297 prompt_eval_duration=110000000 eval_count=21 eval_duration=897000000 message=Message(role='assistant', content='已为您打开了QQ。您可以开始使用它了。如果您有任何问题或需要帮助，请告诉我。', images=None, tool_calls=None)


Alice: 已为您打开了QQ。您可以开始使用它了。如果您有任何问题或需要帮助，请告诉我。


2025-01-13 10:36:50.565 | INFO     | ollama_qw:chat:25 - input message: {'role': 'user', 'content': '打开百度网盘'}
2025-01-13 10:36:51.671 | INFO     | ollama_qw:chat:32 - model response is: model='qwen2.5:3b' created_at='2025-01-13T02:36:51.6711241Z' done=True done_reason='stop' total_duration=1100142800 load_duration=30202400 prompt_eval_count=331 prompt_eval_duration=84000000 eval_count=23 eval_duration=962000000 message=Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='run_software', arguments={'software_name': '百度网盘'}))])
2025-01-13 10:36:51.674 | INFO     | ollama_qw:chat:38 - function name: run_software
2025-01-13 10:36:51.675 | INFO     | ollama_qw:chat:39 - function args: {'software_name': '百度网盘'}
2025-01-13 10:36:51.678 | INFO     | ollama_qw:chat:48 - tool response is: {'role': 'tool', 'name': 'run_software', 'content': None}


Running software 百度网盘


2025-01-13 10:36:52.853 | INFO     | ollama_qw:chat:55 - model response is: model='qwen2.5:3b' created_at='2025-01-13T02:36:52.8523334Z' done=True done_reason='stop' total_duration=1170953800 load_duration=31050700 prompt_eval_count=371 prompt_eval_duration=105000000 eval_count=23 eval_duration=1003000000 message=Message(role='assistant', content='我已经为您打开了百度网盘。您可以开始使用它了。如果您有任何问题或需要帮助，请告诉我。', images=None, tool_calls=None)


Alice: 我已经为您打开了百度网盘。您可以开始使用它了。如果您有任何问题或需要帮助，请告诉我。
